In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [21]:
a,b = mnist.train.next_batch(1)

array([[[[0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         [0.        ],
         

In [74]:
class CNN:
    def __init__(self, sess, h=28, w=28):
        self.size_h = h
        self.size_w = w
        self.sess = sess
        self.model = self.make_model()
    
    def make_model(self):
        # 모델
        self.observation = tf.placeholder(shape=[None, self.size_h, self.size_w, 1], dtype=tf.float32) # 이미지 데이터
        self.label = tf.placeholder(shape=[None, 10], dtype=tf.int16) # 라벨 데이터
        
        self.w_in = tf.Variable(tf.random_normal([5,5,1,8], stddev=.01)) # conv 1 가중치
        self.l1 = tf.nn.conv2d(self.observation, self.w_in, strides=[1,1,1,1], padding='SAME')
        self.l1 = tf.nn.relu(self.l1)
        self.l1 = tf.nn.max_pool(self.l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        
        self.w_out = tf.Variable(tf.random_normal(shape=[self.size_w//2*self.size_h//2*8, 10], stddev=.01))
        self.b = tf.Variable(tf.random_normal([10]))
        self.h_flat = tf.reshape(self.l1, [-1, self.size_w//2*self.size_h//2*8])
        
        self.output = tf.matmul(self.h_flat, self.w_out) + self.b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.output, labels=self.label))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=.001).minimize(self.cost)
        print('> 모델 생성 완료')
        
    def train(self, d, y):
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict={self.observation:d, self.label:y})
        reduced = np.mean(cost)
        return reduced
    
    def test(self, d, y):
        out = self.sess.run(self.output, feed_dict={self.observation:d})
        idx = np.argmax(out, axis=1)
        one_hot = np.zeros_like(out)
        cnt = 0
        for i, arr in enumerate(one_hot):
            arr[idx[i]] = 1
            if(np.all(arr == y[i])):
                cnt += 1
        
        accuracy = cnt / len(d)
        return accuracy

In [77]:
with tf.Session() as sess:
    model = CNN(sess)
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(1000):
        data, Y = mnist.train.next_batch(32)
        cost = model.train(data, Y)
        if(i % 100 == 0):
            print(i, ' cost :',  cost)
            
    for i in range(1000):
        data, Y = mnist.test.next_batch(32)
        acc = model.test(data, Y)
        if(i % 10 == 0):
            print(i, '정확도 :', acc)

> 모델 생성 완료
0  cost : 2.7561512
100  cost : 0.5166166
200  cost : 0.25218165
300  cost : 0.5556947
400  cost : 0.52000713
500  cost : 0.13591866
600  cost : 0.62180495
700  cost : 0.40368283
800  cost : 0.42407036
900  cost : 0.1936966
0 정확도 : 0.9375
10 정확도 : 0.96875
20 정확도 : 0.9375
30 정확도 : 0.90625
40 정확도 : 0.96875
50 정확도 : 0.96875
60 정확도 : 1.0
70 정확도 : 0.96875
80 정확도 : 0.84375
90 정확도 : 0.9375
100 정확도 : 0.96875
110 정확도 : 0.96875
120 정확도 : 0.84375
130 정확도 : 0.90625
140 정확도 : 1.0
150 정확도 : 0.9375
160 정확도 : 0.875
170 정확도 : 0.875
180 정확도 : 0.90625
190 정확도 : 0.875
200 정확도 : 0.96875
210 정확도 : 0.9375
220 정확도 : 0.78125
230 정확도 : 0.96875
240 정확도 : 0.96875
250 정확도 : 1.0
260 정확도 : 1.0
270 정확도 : 0.90625
280 정확도 : 0.875
290 정확도 : 0.96875
300 정확도 : 0.9375
310 정확도 : 0.875
320 정확도 : 0.90625
330 정확도 : 0.90625
340 정확도 : 0.96875
350 정확도 : 0.84375
360 정확도 : 0.9375
370 정확도 : 1.0
380 정확도 : 0.9375
390 정확도 : 0.875
400 정확도 : 0.90625
410 정확도 : 0.96875
420 정확도 : 0.90625
430 정확도 : 0.96875
440 정확도 : 0.9375
450 정확도